In [ ]:
import re
import os
import glob

def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    content = content.replace('\n', ' ')
    return content

def parse_ann_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 2):
            target_info = lines[i].strip().split('\t')
            polarity_info = lines[i + 1].strip().split(' ')

            position_info = target_info[1].split()
            data.append({
                'target': target_info[2],
                'start': int(position_info[1]),
                'end': int(position_info[2]),
                'aspect': position_info[0],
                'polarity': polarity_info[2]
            })
    return data

def get_sentence(text, start, end):
    sentence_start = max(text.rfind('.', 0, start), text.rfind('!', 0, start), text.rfind('?', 0, start)) + 1
    sentence_end = -1
    for char in ['.', '!', '?']:
        end_index = text.find(char, end)
        if end_index != -1 and (sentence_end == -1 or end_index < sentence_end):
            sentence_end = end_index
    if sentence_end == -1:
        sentence_end = len(text)
    sentence = text[sentence_start:sentence_end].strip()
    return sentence


def convert_to_target_format(text, ann_data):
    target_format_data = []
    for data in ann_data:
        sentence = get_sentence(text, data['start'], data['end'])
        sentence = sentence.replace(data['target'], '$T$')
        target = data['target']
        aspect = data['aspect']
        polarity = 1 if data['polarity'] == 'positive' else -1 if data['polarity'] == 'negative' else 0
        target_format_data.append((sentence, target, aspect, polarity))
    return target_format_data

def write_to_seg_file(file_path, target_format_data):
    with open(file_path, 'w') as file:
        for i, data in enumerate(target_format_data):
            file.write('\n'.join(map(str, data)))
            if i < len(target_format_data) - 1:
                file.write('\n')


def process_files(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    ann_files = glob.glob(os.path.join(input_folder, "*.ann"))

    for ann_file in ann_files:
        txt_file = os.path.splitext(ann_file)[0] + ".txt"
        seg_file = os.path.splitext(ann_file)[0] + ".seg"
        seg_file = seg_file.replace(input_folder, output_folder)

        text_content = read_txt_file(txt_file)
        ann_data = parse_ann_file(ann_file)
        target_format_data = convert_to_target_format(text_content, ann_data)
        write_to_seg_file(seg_file, target_format_data)

if __name__ == '__main__':
    from google.colab import drive
    drive.mount('/content/drive')
    input_folder = "ann"
    output_folder = "/content/drive/MyDrive/gp_529/seg2"

    process_files(input_folder, output_folder)


Mounted at /content/drive
